In [ ]:
!pip install -q -U keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
!cd drive/MyDrive/OwlDetection/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adadelta, Adamax
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from kerastuner import RandomSearch


img_height, img_width = 224, 224

# Set up the directories
train_dir = '/content/drive/MyDrive/OwlDetection/train'
validation_dir = '/content/drive/MyDrive/OwlDetection/valid'
test_dir = '/content/drive/MyDrive/OwlDetection/test'

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: validation and test data should not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=64
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=64
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    shuffle=False
)

num_classes = train_generator.num_classes

# Function to create the model, now takes hp object for hyperparameters
def create_model(hp):
    model = Sequential()

    # Optimizing the number of convolutional layers
    for i in range(hp.Int('num_layers', 2, 4)):  # Number of layers is between 2 and 4
        model.add(Conv2D(hp.Choice(f'filters_{i}', [32, 64, 128, 256]),
                         (3, 3), padding='same',
                         input_shape=(img_height, img_width, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(256, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Tuner for optimizer
    hp_optimizer = hp.Choice('optimizer', ['adam', 'adadelta', 'adamax'])
    if hp_optimizer == 'adam':
        optimizer = Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))
    elif hp_optimizer == 'adadelta':
        optimizer = Adadelta(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))
    else:  # hp_optimizer == 'adamax'
        optimizer = Adamax(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Setting up Keras Tuner random search
tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=5,  # Adjust number of trials
    executions_per_trial=1,
    directory='/content/drive/MyDrive/OwlDetection/hyperparameter_tuning',
    project_name='owl_detection_tuning'
)

# Start search
tuner.search(
    train_generator,
    epochs=65,
    validation_data=validation_generator,
    callbacks=[early_stopping]  # Early stopping callback
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of layers is {best_hps.get('num_layers')}.
The optimal filters per layer are {', '.join(str(best_hps.get(f'filters_{i}')) for i in range(best_hps.get('num_layers')))}.
The optimal activation function is {best_hps.get('activation')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
The optimal optimizer is {best_hps.get('optimizer')}.
""")

# Retrain the model with the best hyperparameters and evaluate on the test data if needed


Found 923 images belonging to 6 classes.
Found 30 images belonging to 6 classes.
Found 30 images belonging to 6 classes.

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
32                |32                |filters_0
128               |128               |filters_1
adamax            |adamax            |optimizer
0.0001            |0.0001            |learning_rate

Epoch 1/65
15/15 [==============================] - 335s 22s/step - loss: 6.9853 - accuracy: 0.1744 - val_loss: 6.7411 - val_accuracy: 0.2333
Epoch 2/65
15/15 [==============================] - 16s 1s/step - loss: 6.7345 - accuracy: 0.1950 - val_loss: 6.6565 - val_accuracy: 0.1667
Epoch 3/65
15/15 [==============================] - 16s 1s/step - loss: 6.6354 - accuracy: 0.1831 - val_loss: 6.5826 - val_accuracy: 0.1667
Epoch 4/65
15/15 [==============================] - 16s 1s/step - loss: 6.5401 - accuracy: 0.2069 - val_loss: 6.5152 - val_accuracy

KeyboardInterrupt: 